In [1]:
# Installation
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install tqdm
!pip install matplotlib
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: tensorflow-serving-api 2.2.0 has a non-standard dependency specifier grpcio>=1.0<2. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-serving-api or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: tensorflow-serving-api 2.2.0 has a non-standard dependency specifier grpcio>=1.0<2. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-serving-api or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-pack

In [2]:
# Generate requirements.txt
!pip freeze > requirements.txt

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sksurv.column import encode_categorical
from sksurv.preprocessing import OneHotEncoder as SurvivalOneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sklearn.impute import SimpleImputer
import os
from tqdm import tqdm
import clean_data


current_dir = os.getcwd().replace("\\", "/")
df = pd.read_csv(current_dir + "/data/train.csv")

df = clean_data.clean_data(df)
# Überprüfen, ob 'efs' jetzt nur 0 und 1 enthält
print("Eindeutige Werte in 'efs':", df['efs'].unique())

# Zielvariable definieren (Ereignis und Zeit)
try:
    y = Surv.from_arrays(event=df['efs'], time=df['efs_time'])
except ValueError as e:
    print("Fehler beim Erstellen des Surv-Objekts:", e)
    print("Datentyp von 'efs':", df['efs'].dtype)
    print("Eindeutige Werte in 'efs':", df['efs'].unique())


# Entferne Zielvariablen aus den Features
X = df.drop(columns=['efs', 'efs_time'])

# Numerische und kategoriale Spalten identifizieren
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Alle kategorialen Spalten in Strings umwandeln
X[cat_cols] = X[cat_cols].astype(str)

# Fehlende Werte ersetzen
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

# One-Hot-Encoding für kategoriale Variablen
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_cat = encoder.fit_transform(X[cat_cols])
X_cat = pd.DataFrame(X_cat, columns=encoder.get_feature_names_out(cat_cols))

# Standardisierung der numerischen Variablen
scaler = StandardScaler()
X_num = pd.DataFrame(scaler.fit_transform(X[num_cols]), columns=num_cols)

# Kombinieren von numerischen und kodierten kategorialen Features
X_processed = pd.concat([X_num, X_cat], axis=1)

# Survival-Modell erstellen
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=10, random_state=42)

# Fortschrittsbalken hinzufügen
for _ in tqdm(range(1), desc="Training des Modells"):
    rsf.fit(X_processed, y)

# Vorhersage des "Risk Scores" mit Fortschrittsbalken
predictions = []
for i in tqdm(range(len(X_processed)), desc="Vorhersagen berechnen"):
    predictions.append(rsf.predict(X_processed.iloc[i:i+1])[0])

# Speichern der Ergebnisse
results = pd.DataFrame({'ID': df.index, 'prediction': predictions})

[0. 1.]
Eindeutige Werte in 'efs' nach Bereinigung: [0 1]
Kategorische Spalte 'dri_score': Fehlende Werte mit häufigstem Wert auffüllen.
Kategorische Spalte 'psych_disturb': Fehlende Werte mit häufigstem Wert auffüllen.
Kategorische Spalte 'cyto_score': Fehlende Werte mit häufigstem Wert auffüllen.
Kategorische Spalte 'diabetes': Fehlende Werte mit häufigstem Wert auffüllen.
Numerische Spalte 'hla_match_c_high': Fehlende Werte mit Median auffüllen.
Numerische Spalte 'hla_high_res_8': Fehlende Werte mit Median auffüllen.
Kategorische Spalte 'arrhythmia': Fehlende Werte mit häufigstem Wert auffüllen.
Numerische Spalte 'hla_low_res_6': Fehlende Werte mit Median auffüllen.
Kategorische Spalte 'vent_hist': Fehlende Werte mit häufigstem Wert auffüllen.
Kategorische Spalte 'renal_issue': Fehlende Werte mit häufigstem Wert auffüllen.
Kategorische Spalte 'pulm_severe': Fehlende Werte mit häufigstem Wert auffüllen.
Numerische Spalte 'hla_high_res_6': Fehlende Werte mit Median auffüllen.
Kategori

Vorhersagen berechnen: 100%|█████████████| 28800/28800 [03:34<00:00, 134.00it/s]


In [3]:
results.head()

,ID,prediction
0,0,6.778629
1,1,51.584190
2,2,12.277942
3,3,33.991870
4,4,23.372414
